In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample

import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
import sklearn
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,Concatenate,concatenate, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
#from tensorflow.keras.utils import np_utils
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.regularizers import l2

In [2]:
tensorflow.keras.backend.clear_session()

In [3]:
model_spo2 = Sequential()
model_spo2.add(BatchNormalization(input_shape=(88,1)))
model_spo2.add(Conv1D(6, kernel_size=(25),padding='same'))
model_spo2.add(Activation("relu"))
#model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Conv1D(50, (10),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(Conv1D(30, (15),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(BatchNormalization())
model_spo2.add(Flatten())
model_spo2.add(Dropout(0.25))
model_spo2.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_spo2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 88, 1)             4         
_________________________________________________________________
conv1d (Conv1D)              (None, 88, 6)             156       
_________________________________________________________________
activation (Activation)      (None, 88, 6)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 88, 50)            3050      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 44, 50)            0         
_________________________________________________________________
activation_1 (Activation)    (None, 44, 50)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 44, 30)            2

In [4]:
model_spo2.load_weights('./best_weights_spo2_3_312layer.hdf5')

In [5]:
weight_spo2=model_spo2.get_weights()

In [6]:
second_input = Input(shape=(88,1))
second_batch = BatchNormalization(weights=weight_spo2[0:4],trainable=False)(second_input)
second_conv= Conv1D(6,kernel_size=(25),padding='same',weights=[weight_spo2[4],weight_spo2[5]],trainable=False)(second_batch)
second_act=Activation("relu")(second_conv)
second_conv2=Conv1D(50,kernel_size=(10),padding='same',weights=[weight_spo2[6],weight_spo2[7]],trainable=False)(second_act)
second_maxpool2=MaxPooling1D(pool_size=(2))(second_conv2)
second_act2=Activation("relu")(second_maxpool2)
second_conv3=Conv1D(30,kernel_size=(15),padding='same',weights=[weight_spo2[8],weight_spo2[9]],trainable=False)(second_maxpool2)
second_maxpool3=MaxPooling1D(pool_size=(2))(second_conv3)
second_act3=Activation("relu")(second_maxpool3)
second_batch2 = BatchNormalization(weights=weight_spo2[10:14],trainable=False)(second_act3)
second_final = Flatten()(second_batch2)


In [7]:
model_abdo = Sequential()
model_abdo.add(BatchNormalization(input_shape=(1408,1)))
model_abdo.add(Conv1D(3, kernel_size=(100),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Conv1D(50, (10)))
model_abdo.add(MaxPooling1D(pool_size=(2),strides=2))
model_abdo.add(Activation("relu"))
model_abdo.add(Conv1D(30, (30)))
model_abdo.add(MaxPooling1D(pool_size=(2)))
model_abdo.add(Activation("relu"))
model_abdo.add(BatchNormalization())
model_abdo.add(Flatten())
model_abdo.add(Dropout(0.25))
model_abdo.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_abdo.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 1408, 1)           4         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 655, 3)            303       
_________________________________________________________________
activation_6 (Activation)    (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 327, 3)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 159, 50)           0         
_________________________________________________________________
activation_7 (Activation)    (None, 159, 50)          

In [8]:
model_abdo.load_weights('./best_weights_abdo_32layer.hdf5')

In [9]:
weight_abdo=model_abdo.get_weights()

In [10]:
third_input = Input(shape=(1408,1))
third_batch = BatchNormalization(weights=weight_abdo[0:4],trainable=False)(third_input)
third_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_abdo[4],weight_abdo[5]],trainable=False)(third_batch)
third_act=Activation("relu")(third_conv)
third_maxpool= MaxPooling1D(pool_size=(2),strides=2)(third_act)
third_conv2=Conv1D(50,kernel_size=(10),weights=[weight_abdo[6],weight_abdo[7]],trainable=False)(third_maxpool)
third_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(third_conv2)
third_act2=Activation("relu")(third_maxpool2)
third_conv3=Conv1D(30,kernel_size=(30),weights=[weight_abdo[8],weight_abdo[9]],trainable=False)(third_act2)
third_maxpool3=MaxPooling1D(pool_size=(2))(third_conv3)
third_act3=Activation("relu")(third_maxpool3)
third_batch2 = BatchNormalization(weights=weight_abdo[10:14],trainable=False)(third_act3)
third_flatten = Flatten()(third_batch2)
third_dropout=Dropout(0.33)(third_flatten)
third_final=Flatten()(third_dropout)

In [11]:
merge_layer=concatenate([second_final, third_final])

In [12]:
merge_batch1 = BatchNormalization()(merge_layer)
merge_drop2 = Dropout(0.25)(merge_batch1)
merge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)
merge_output= Activation("softmax")(merge_dense2)

In [13]:
model_merge_spo_abdo = Model(inputs=[second_input, third_input], outputs=merge_output)

In [14]:
model_merge_spo_abdo.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 1408, 1)      4           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 655, 3)       303         batch_normalization_6[0][0]      
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 655, 3)       0           conv1d_9[0][0]                   
______________________________________________________________________________________________

In [15]:
batch_size = 64
epochs = 500

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model_merge_spo_abdo.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
batch_size = 64
epochs = 500

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model_merge_spo_abdo.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [17]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',



valid_features1=np.zeros((1,1408))
valid_features2=np.zeros((1,1408))
valid_features3=np.zeros((1,1408))
test_features1=np.zeros((1,1408))
test_features2=np.zeros((1,1408))
test_features3=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        spo2_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_valid.mat')
        spo2_valid = np.array(spo2_valid['spo2_valid'])
        
        abdo_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_valid.mat')
        abdo_valid = np.array(abdo_valid['abdo_valid'])
        spo2_valid_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_valid_labels.mat')
        spo2_valid_labels = np.array(spo2_valid_labels['class_valid'])
        valid_features1=np.append(valid_features1,spo2_valid,axis=0)
        
        valid_features3=np.append(valid_features3,abdo_valid,axis=0)
        valid_labels=np.append(valid_labels,spo2_valid_labels)
       
        
        spo2_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_test.mat')
        spo2_test = np.array(spo2_test['spo2_test'])
       
        abdo_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_test.mat')
        abdo_test = np.array(abdo_test['abdo_test'])
        spo2_test_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_test_labels.mat')
        spo2_test_labels = np.array(spo2_test_labels['class_test'])
        test_features1=np.append(test_features1,spo2_test,axis=0)
        
        test_features3=np.append(test_features3,abdo_test,axis=0)
        test_labels=np.append(test_labels,spo2_test_labels)
        
spo2_valid=valid_features1[1:,0::16]

abdo_valid=valid_features3[1:]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()

spo2_test=test_features1[1:,0::16]

abdo_test=test_features3[1:]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [18]:
train_features1=np.zeros((1,1408))
train_features2=np.zeros((1,1408))
train_features3=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        spo2_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_train.mat')
        spo2_train = np.array(spo2_train['spo2_train'])
        
        abdo_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_abdo_train.mat')
        abdo_train = np.array(abdo_train['abdo_train'])
        spo2_train_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_train_labels.mat')
        spo2_train_labels = np.array(spo2_train_labels['class_train'])
        train_features1=np.append(train_features1,spo2_train,axis=0)
        
        train_features3=np.append(train_features3,abdo_train,axis=0)
        train_labels=np.append(train_labels,spo2_train_labels)
        
spo2_train=train_features1[1:,0::16]

abdo_train=train_features3[1:]
train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [19]:
train_features1=0
train_features2=0
train_features3=0
valid_features1=0
valid_features2=0
valid_features3=0
test_features1=0
test_features2=0
test_features3=0

In [20]:
abdo_train_mean=np.mean(abdo_train)
abdo_train_std=np.std(abdo_train)

In [21]:
for i in range(abdo_train.shape[0]):
    abdo_train[i,:]=(abdo_train[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_valid.shape[0]):
    abdo_valid[i,:]=(abdo_valid[i,:]-abdo_train_mean)/abdo_train_std
    
for i in range(abdo_test.shape[0]):
    abdo_test[i,:]=(abdo_test[i,:]-abdo_train_mean)/abdo_train_std
    

abdo_train=np.expand_dims(abdo_train, axis=2)
abdo_valid=np.expand_dims(abdo_valid, axis=2)
abdo_test=np.expand_dims(abdo_test, axis=2)

In [22]:
spo2_train_mean=np.mean(spo2_train)
spo2_train_std=np.std(spo2_train)

In [23]:
for i in range(spo2_train.shape[0]):
    spo2_train[i,:]=(spo2_train[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_valid.shape[0]):
    spo2_valid[i,:]=(spo2_valid[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_test.shape[0]):
    spo2_test[i,:]=(spo2_test[i,:]-spo2_train_mean)/spo2_train_std

spo2_train=np.expand_dims(spo2_train, axis=2)
spo2_valid=np.expand_dims(spo2_valid, axis=2)
spo2_test=np.expand_dims(spo2_test, axis=2)

In [24]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [25]:
num_classes=2
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [26]:
best_weights_filepath_merge = './best_weights_fusion_spo_abdo_9_2.hdf5'
mcp_merge = ModelCheckpoint(best_weights_filepath_merge, monitor="val_accuracy",
                      save_best_only=True, save_weights_only=False)
            
history = model_merge_spo_abdo.fit([spo2_train, abdo_train], y_train_wide,
         batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([spo2_valid, abdo_valid], y_valid_wide),
          callbacks=[mcp_merge])

Epoch 1/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.1333 - accuracy: 0.9620 - val_loss: 0.0463 - val_accuracy: 0.9958
Epoch 2/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0924 - accuracy: 0.9759 - val_loss: 0.0424 - val_accuracy: 0.9961
Epoch 3/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0910 - accuracy: 0.9758 - val_loss: 0.0413 - val_accuracy: 0.9958
Epoch 4/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0894 - accuracy: 0.9760 - val_loss: 0.0392 - val_accuracy: 0.9965
Epoch 5/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0884 - accuracy: 0.9759 - val_loss: 0.0382 - val_accuracy: 0.9959
Epoch 6/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0874 - accuracy: 0.9758 - val_loss: 0.0374 - val_accuracy: 0.9957
Epoch 7/500
12593/12593 [==============================] - 57s 4ms/step - loss: 0.0865 - accuracy: 0.9758 - val_

12593/12593 [==============================] - 54s 4ms/step - loss: 0.0742 - accuracy: 0.9761 - val_loss: 0.0236 - val_accuracy: 0.9964
Epoch 57/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0739 - accuracy: 0.9763 - val_loss: 0.0242 - val_accuracy: 0.9961
Epoch 58/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0740 - accuracy: 0.9759 - val_loss: 0.0230 - val_accuracy: 0.9965
Epoch 59/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0736 - accuracy: 0.9761 - val_loss: 0.0232 - val_accuracy: 0.9966
Epoch 60/500
12593/12593 [==============================] - 57s 5ms/step - loss: 0.0739 - accuracy: 0.9760 - val_loss: 0.0229 - val_accuracy: 0.9964
Epoch 61/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0739 - accuracy: 0.9760 - val_loss: 0.0234 - val_accuracy: 0.9966
Epoch 62/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0739 - accuracy: 0.9760 - val_loss: 

Epoch 111/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0719 - accuracy: 0.9759 - val_loss: 0.0209 - val_accuracy: 0.9963
Epoch 112/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0717 - accuracy: 0.9759 - val_loss: 0.0207 - val_accuracy: 0.9965
Epoch 113/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0723 - accuracy: 0.9757 - val_loss: 0.0200 - val_accuracy: 0.9964
Epoch 114/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0716 - accuracy: 0.9759 - val_loss: 0.0206 - val_accuracy: 0.9962
Epoch 115/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0716 - accuracy: 0.9758 - val_loss: 0.0209 - val_accuracy: 0.9965
Epoch 116/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0716 - accuracy: 0.9760 - val_loss: 0.0202 - val_accuracy: 0.9967
Epoch 117/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0714 - accuracy:

Epoch 220/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0707 - accuracy: 0.9755 - val_loss: 0.0182 - val_accuracy: 0.9965
Epoch 221/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0713 - accuracy: 0.9754 - val_loss: 0.0181 - val_accuracy: 0.9965
Epoch 222/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0700 - accuracy: 0.9757 - val_loss: 0.0175 - val_accuracy: 0.9966
Epoch 223/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0703 - accuracy: 0.9757 - val_loss: 0.0184 - val_accuracy: 0.9965
Epoch 224/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0699 - accuracy: 0.9762 - val_loss: 0.0185 - val_accuracy: 0.9964
Epoch 225/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0702 - accuracy: 0.9757 - val_loss: 0.0183 - val_accuracy: 0.9964
Epoch 226/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0700 - accuracy:

Epoch 329/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0705 - accuracy: 0.9755 - val_loss: 0.0175 - val_accuracy: 0.9965
Epoch 330/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0705 - accuracy: 0.9751 - val_loss: 0.0169 - val_accuracy: 0.9966
Epoch 331/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0697 - accuracy: 0.9756 - val_loss: 0.0173 - val_accuracy: 0.9967
Epoch 332/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0703 - accuracy: 0.9755 - val_loss: 0.0172 - val_accuracy: 0.9965
Epoch 333/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0698 - accuracy: 0.9757 - val_loss: 0.0176 - val_accuracy: 0.9963
Epoch 334/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0700 - accuracy: 0.9756 - val_loss: 0.0173 - val_accuracy: 0.9965
Epoch 335/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0695 - accuracy:

Epoch 438/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0707 - accuracy: 0.9751 - val_loss: 0.0168 - val_accuracy: 0.9964
Epoch 439/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0696 - accuracy: 0.9757 - val_loss: 0.0178 - val_accuracy: 0.9965
Epoch 440/500
12593/12593 [==============================] - 55s 4ms/step - loss: 0.0696 - accuracy: 0.9756 - val_loss: 0.0168 - val_accuracy: 0.9965
Epoch 441/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0697 - accuracy: 0.9754 - val_loss: 0.0175 - val_accuracy: 0.9963
Epoch 442/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0693 - accuracy: 0.9756 - val_loss: 0.0171 - val_accuracy: 0.9965
Epoch 443/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0701 - accuracy: 0.9753 - val_loss: 0.0172 - val_accuracy: 0.9964
Epoch 444/500
12593/12593 [==============================] - 57s 5ms/step - loss: 0.0693 - accuracy:

In [28]:
y_pred = model_merge_spo_abdo.predict([spo2_test, abdo_test])
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[50175,   193],
       [   13,  1355]], dtype=int64)

In [29]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9960182464821401

In [30]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.9904970760233918

In [31]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9961682020330368

# load saved weights

In [32]:
model_merge_spo_abdo.load_weights('./best_weights_fusion_spo_abdo_9_2.hdf5')

In [33]:
y_pred = model_merge_spo_abdo.predict([spo2_test, abdo_test])
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[50183,   185],
       [   15,  1353]], dtype=int64)

In [34]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9961342198855729

In [35]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.9890350877192983

In [36]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9963270330368488